In [1]:
from BrattEssay import load_bratt_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings

settings = Settings()

folder = settings.data_directory + "SkinCancer/EBA1415_Merged/"
#folder = settings.data_directory + "CoralBleaching/BrattData/EBA1415_Merged/"
essays = load_bratt_essays(folder)

wd_sent_freq = defaultdict(int)
all_codes = set()
#Stores all words for the spelling corrector
words = []
all_sentences = []
sentencesForCode = defaultdict(list)
for essay in essays:
    for sentence in essay.tagged_sentences:
        wdsInSent = set()
        codes4sentence = set()
        sent = []
        for w, tags in sentence:
            words.append(w)
            all_codes.update(tags)
            codes4sentence.update(tags)
            if w not in wdsInSent:
                wdsInSent.add(w)
                wd_sent_freq[w] += 1
            sent.append(w)
        all_sentences.append(sent)
        for code in codes4sentence:
            sentencesForCode[code].append(sentence)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
1107 files found
Skipping /Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/EBA1415_Merged/EBA1415_BGJD_1_SC_ES-05728.ann file as .txt file is no essay'
Skipping /Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/EBA1415_Merged/EBA1415_BGJD_1_SC_ES-5726_9.ann file as .txt file is no essay.'
Skipping /Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/EBA1415_Merged/EBA1415_KYLS_6_SC_ES-05674.ann file as .txt file is no essay'
Skipping /Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/EBA1415_Merged/EBA1415_LRJE_7_SC_ES-05142.ann file as .txt file is no essay'
Skipping /Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/EBA1415_Merged/EBA1415_RDCS_1_SC_ES-04696.ann file as .txt file is no essay//'
Skipping /Users/simon.hughes/Google Drive/P

### Compute Stats over the Essays

In [39]:
wd_counts = []
sent_counts = []
concept_codes = []
cr_concept_codes = []
sent_multi_word_tags = {}
sent_codes = []
sent_cr_codes = []

num_sents = 0
un_wd_counts = []
for e_ix, essay in enumerate(essays):
    wds = 0
    un_words = set()
    for i, sentence in enumerate(essay.tagged_sentences):
        num_sents += 1
        sent_tags = set()
        sent_cr_tags = set()
        for w, tags in sentence:
            un_words.add(w)
            wds += 1
            ccodes = [t for t in tags if t[0].isdigit()]
            if ccodes:
                sent_tags.update(ccodes)
                concept_codes.append(ccodes)
                if len(ccodes) > 1:
                    sent_multi_word_tags[(e_ix, i)] = [(w,[tag for tag in t if tag[0].isdigit()]) for w,t in sentence]
            cr_codes = [t for t in tags if t[0].isdigit() or t == "Causer" or t == "Result" or t == "explicit"]
            if cr_codes:
                cr_concept_codes.append(cr_codes)
                sent_cr_tags.update(cr_codes)
        if len(sent_tags) > 0:
            sent_codes.append(sent_tags)
            if len(sent_cr_tags) > 0:
                sent_cr_codes.append(sent_cr_tags)
                
    un_wd_counts.append(len(un_words))
    sent_counts.append(len(essay.tagged_sentences))
    wd_counts.append(wds)

In [26]:
print len(sent_multi_word_tags), num_sents
sent_multi_word_tags.items()[0]

1 10670


((440, 9),
 [('Sunburn', ['5']),
  ('happens', []),
  ('when', []),
  ('the', []),
  ('body', []),
  ('directs', []),
  ('blood', []),
  ('to', []),
  ('try', []),
  ('to', []),
  ('repair', ['6']),
  ('or', ['6']),
  ('remove', ['6']),
  ('cells', ['6', '4']),
  ('damaged', ['6', '4']),
  ('by', []),
  ('the', []),
  ('sun', ['2']),
  ('.', [])])

In [27]:
len(sent_codes), len([tags for tags in sent_codes if len(tags) > 1])

(6671, 4555)

### Essay Length Statistics

In [28]:
import numpy as np
print np.mean(wd_counts), np.median(wd_counts), np.min(wd_counts), np.max(wd_counts), np.std(wd_counts)
print np.mean(sent_counts), np.median(sent_counts), np.min(sent_counts), np.max(sent_counts), np.std(sent_counts)

166.267463235 157.0 4 479 82.449645775
9.80698529412 9.0 1 36 5.02616928448


### Vocabulary Size

In [29]:
np.min(un_wd_counts), np.max(un_wd_counts), np.mean(un_wd_counts), np.median(un_wd_counts), np.std(un_wd_counts)

(4, 215, 90.163602941176464, 88.0, 36.152364440875424)

In [30]:
from IterableFP import flatten
unique = set(flatten(concept_codes))
unique

{'1', '11', '12', '2', '3', '4', '5', '50', '6'}

### How Many Tagged Words Have Multiple Codes?

In [31]:
print len(concept_codes), sum(wd_counts), round(len(concept_codes) / float(sum(wd_counts)),2)
multiple = [tags for tags in concept_codes if len(tags) > 1]
print len(multiple), len(multiple) / float(len(concept_codes))
multiple

43024 180899 0.24
2 4.64856824098e-05


[['6', '4'], ['6', '4']]

In [32]:
unique_cr = set(flatten(cr_concept_codes))
unique_cr

{'1',
 '11',
 '12',
 '2',
 '3',
 '4',
 '5',
 '50',
 '6',
 'Causer',
 'Result',
 'explicit'}

In [38]:
print len(cr_concept_codes), sum(wd_counts), round(len(cr_concept_codes) / float(sum(wd_counts)),2)
multiple = [tags for tags in cr_concept_codes if len(tags) > 1]
print len(multiple), len(multiple) / float(len(cr_concept_codes))

50260 180899 0.28
34170 0.679864703542


### What Proportion of Sentences With Codes Have Multiple Codes?

In [34]:
num_sents = float(num_sents)
len(sent_codes) / num_sents
num_multiple_codes = len([tags for tags in sent_codes if len(tags) > 1])
num_multiple_codes / float(len(sent_codes))

0.6828061759856093

In [40]:
len(sent_cr_codes) / num_sents
num_multiple_codes = len([tags for tags in sent_cr_codes if len(tags) > 1])
num_multiple_codes / float(len(sent_cr_codes))

0.7507120371758357

### Conditional Probabilities of Codes

In [35]:
from collections import defaultdict
priors = defaultdict(float)
joints = defaultdict(float)

for sent in sent_codes:
    for a in sorted(sent):
        priors[a] += 1
        for b in sorted(sent):
            if b >= a:
                break
            joints[(b,a)] +=1

conditional = {}
for a, cnt in priors.items():
    for b in priors.keys():
        if a == b:
            continue
        """ p(A/B) = p(B/A)p(A) / p(B) """
        """ p(A/B) = p(B/A)p(A) """
        if a < b:
            joint = joints[(a,b)]
        else:
            joint = joints[(b,a)]
        conditional[(a,b)] = joint / priors[b]
    
lifts = {}
total = float(sum(joints.values()))
totalPrior = float(sum(priors.values()))
for (a,b),cnt in joints.items():
    joint = cnt / total
    pA = priors[a] / totalPrior
    pB = priors[b] / totalPrior
    lift = joint / (pA * pB)
    if lift:
        lifts[(a,b)] = lift

In [42]:
def get_num(a):
    s = ""
    for c in a:
        if c.isdigit():
            s += c
    return int(s)

consec_pmi = {}
for (a,b), lift in lifts.items():
    ia = get_num(a)
    ib = get_num(b)
    diff = abs(ia-ib)
    pmi = np.log(lift)
    if "Coral" in folder:
        if diff == 1 and b != "6" and b != "5b":
            consec_pmi[(a,b)] = pmi
        elif a =="5" and b =="5b":
            consec_pmi[(a,b)] = pmi
        elif a =="14" and b =="5b":
            consec_pmi[(a,b)] = pmi
        elif a =="14" and b =="6":
            consec_pmi[(a,b)] = pmi
        elif a =="50" and b =="7":
            consec_pmi[(a,b)] = pmi
    elif "Skin" in folder:
        if diff == 1:
            consec_pmi[(a,b)] = pmi
        elif a == "12" and b == "6":
            consec_pmi[(a,b)] = pmi
        elif a == "50" and b == "6":
            consec_pmi[(a,b)] = pmi        
            
for k,v in sorted(consec_pmi.items(), key = lambda (k,v): (min(int(k[0]),int(k[1].replace("b",""))))):
    a = k[0]
    b = k[1]
    if len(a) > len(b.replace("b","")):
        a,b = b,a
    print "&", str(a).ljust(5), "&", str(b).ljust(5), "&", str(round(v,2)).rjust(5), "\\\\"

& 1     & 2     &  1.83 \\
& 2     & 3     &  1.19 \\
& 3     & 4     &  1.54 \\
& 4     & 5     &  1.39 \\
& 5     & 6     &  1.69 \\
& 6     & 50    &   1.1 \\
& 6     & 12    &  -1.5 \\
& 11    & 12    &  2.82 \\


In [37]:
np.mean(lifts.values())

2.5111765747794741

In [43]:
np.mean(map(lambda l: np.log(l), lifts.values()))

0.29579903430068333

In [45]:
sorted(lifts.items(), key=lambda (tpl,p):-p)

[(('11', '12'), 16.729862357481394),
 (('11', '3'), 7.405751434489755),
 (('12', '3'), 7.211182655573351),
 (('1', '2'), 6.221040488119156),
 (('5', '6'), 5.424637638305385),
 (('3', '4'), 4.665913991679826),
 (('4', '5'), 4.016418788952101),
 (('4', '6'), 3.317247469708471),
 (('2', '3'), 3.274400630435474),
 (('5', '50'), 3.228817828809839),
 (('1', '50'), 3.0957572838200496),
 (('50', '6'), 2.9981403258405885),
 (('2', '50'), 2.6287234482050525),
 (('3', '50'), 1.939223843652011),
 (('2', '4'), 1.934500132777979),
 (('1', '3'), 1.8280169052807465),
 (('11', '50'), 1.4306591000477737),
 (('4', '50'), 1.3030148061014875),
 (('1', '11'), 1.245350887924594),
 (('3', '6'), 1.1303601304786342),
 (('2', '5'), 1.0909797087629334),
 (('11', '2'), 1.00827032383412),
 (('12', '4'), 0.9886534341480678),
 (('3', '5'), 0.9355091530934038),
 (('1', '5'), 0.7793797541411285),
 (('11', '5'), 0.7639145130541294),
 (('12', '2'), 0.6984659944821878),
 (('1', '4'), 0.6014908602065931),
 (('12', '50'), 0